In [22]:
!pip3 install torchvision

     |████████████████████████████████| 645kB 471kB/s eta 0:00:01
     |████████████████████████████████| 3.9MB 707kB/s eta 0:00:011


In [1]:
from settings import Settings
from utils.datasets import MaestroDataset
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.nn.init
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [2]:
settings = Settings() # Set Extra Settings in settings.py

In [3]:
dataset = MaestroDataset("train", settings=settings)

In [4]:
print('total number : {}'.format(len(dataset)))

total number : 144910


In [5]:
filt, vel = dataset[10]

In [6]:
filt[:64].shape

(64, 128)

In [7]:
class lstm_Model(nn.Module):
    def __init__(self, n_of_frames,n_of_cnn_frames, kernel_size, output_channel1, output_channel2):
        super(lstm_Model, self).__init__()

        self.n_of_frames = settings.length
        self.n_of_cnn_frames = n_of_cnn_frames
        self.kernel_size = kernel_size
        self.output_channel1 = output_channel1
        self.output_channel2 = output_channel2
        self.Rw1 = n_of_cnn_frames - kernel_size + 1
        self.Rh1 = 128 - kernel_size + 1
        self.Rw2 = (self.Rw1//2) - kernel_size + 1
        self.Rh2 = (self.Rh1//2) - kernel_size + 1
        self.Rw = self.Rw2//2
        self.Rh = self.Rh2//2
        self.sigmoid = nn.Sigmoid()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, output_channel1, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(output_channel1, output_channel2, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.LSTM(1*self.Rh*self.Rw, 128)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out, (_, _) = self.layer3(out.view(x.size(0), 1, -1))
        out = self.sigmoid(out)

        return out

    def loss(self, output, target):
        loss = nn.MSELoss(output.view(1, -1), target)

        return loss

In [8]:
cnt=0
model = lstm_Model(settings.length, n_of_cnn_frames, 5, 1, 1)

for filt, vel in dataset:
    cnt+=1
    if cnt>=10: break
    for it in range(0,settings.length):
        frame = filt[it:it+n_of_cnn_frames]
        if len(frame)< n_of_cnn_frames:
            frame=np.append(frame, [np.zeros(128, dtype=bool)]*(n_of_cnn_frames-len(frame)),axis=0)
        print(model(torch.Tensor(frame).view(1,1,n_of_cnn_frames,-1)))

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50

...

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50

tensor([[[0.5002, 0.5011, 0.4983, 0.4981, 0.4961, 0.4949, 0.5071, 0.5033,
          0.5081, 0.4984, 0.5014, 0.5049, 0.5052, 0.5015, 0.4963, 0.4949,
          0.5093, 0.5073, 0.4999, 0.5031, 0.4975, 0.4994, 0.5019, 0.4986,
          0.4995, 0.4974, 0.4942, 0.5005, 0.4960, 0.5008, 0.4922, 0.4998,
          0.4968, 0.4945, 0.5046, 0.5037, 0.5072, 0.4960, 0.5024, 0.5023,
          0.5091, 0.4958, 0.4969, 0.4919, 0.5023, 0.4933, 0.4997, 0.4931,
          0.5023, 0.5020, 0.4946, 0.5003, 0.5001, 0.4969, 0.4998, 0.4976,
          0.5051, 0.5031, 0.5074, 0.5018, 0.5036, 0.4947, 0.5054, 0.5093,
          0.4950, 0.4948, 0.4938, 0.5011, 0.4959, 0.5017, 0.5023, 0.5062,
          0.5086, 0.5009, 0.5011, 0.4953, 0.4997, 0.4997, 0.4923, 0.5064,
          0.5010, 0.4963, 0.4955, 0.4935, 0.4977, 0.4975, 0.4897, 0.5000,
          0.5058, 0.5077, 0.4990, 0.4938, 0.5018, 0.4975, 0.4949, 0.4953,
          0.4996, 0.4994, 0.5008, 0.5084, 0.5016, 0.5020, 0.5046, 0.5025,
          0.5032, 0.4921, 0.4984, 0.50